In [1]:
import torch
import torchvision
from torchvision import transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import numpy as np

import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

import torch.optim as optim
import torch.nn.functional as F


In [2]:
device = torch.device('mps')

mps


In [3]:
train_path = 'fashion_train.npy'
test_path = 'fashion_test.npy'

In [4]:
class CustomDataset(Dataset):
    def __init__(self, npy_file, transform=None):
        self.data = np.load(npy_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        target = self.data[idx, -1]
        image = np.reshape(self.data[idx, :-1], (-1, 28))
       
        sample = {'image': image, 'target': target} 

        if self.transform:
            sample['image'] = self.transform(sample['image'])

        return sample

In [5]:
batch_size=4
norm_metric = 1/255

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize(norm_metric, norm_metric),
     transforms.Grayscale(num_output_channels=1)])
    
train_set = CustomDataset(train_path, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = CustomDataset(test_path, transform=transform)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

classes = {'T-shirt', 'Trouser', 'Pullover', 'Dress', 'Shirt'}

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = Net()
model.to(device)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = Variable(data['image'].to(device)), Variable(data['target'].to(device))
        
        # zero the parameter gradients
        optimizer.zero_grad()

        outputs = model(inputs)

        #compute loss
        loss = criterion(outputs, labels)

        # backpropagate
        loss.backward()

        # update weights
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

: 

: 